# Satellite Vu Python SDK

## Installation

Python 3.8 or higher is required for this package to work. To install the package from PyPi run: `pip install satellitevu`

If the package can't be installed straight off the bat, you may need to install any extra dependencies our sdk needs first. The most common is having to install the appdirs pachage (`pip install appdirs`).

## Setup

To use the package you first need to have access to the Satellite Vu API by creating a client credential set.

1. Log into https://id.qa.satellitevu.com/

2. Click the create client to obtain a client_id and client_secret. The client_secret will only be visible now so copy it and keep it secure!
    - If you already have a client but have forgotten the secret, click Rotate Secret



These will be used to authenticate your access to the API. You can either set them as environment variables in the shell or in your Python script e.g.

**Important: When your client is initially created, you will only have access to the Archive Search. To be able to submit and download orders, please ask one of the Platform team to authorize you for performing these actions.**

In [1]:
# Windows CMD Line

set CLIENT_ID=<client_id>
set CLIENT_SECRET=<client_secret>

SyntaxError: invalid syntax (898400381.py, line 3)

In [ ]:
# Mac/Unix

# This method sets them in your local session i.e. when you restart the terminal you will have to reset this

export CLIENT_ID=<client_id>
export CLIENT_SECRET=<client_secret>

# To add it to your profile permanently, open your bash profile with your favourite editor e.g

nano  ~/.bash_profile # (bash users)
nano ~/.zshrc # (zsh users)

# Then add the export lines as above and save it

: 

In [66]:
import os

# ** Only do this if you're not sharing your scripts or pushing them to Github because we're hardcoding privileged info here! ** 
client_id = "TFlrcIukpDcD2NqwykoL0hOAFzmtZWWC"
client_secret = "KC8rO3uKlyD4PrmAfGioS6Qf_32cBofxlWf6lqW6YV_6SEfaOscXQr7EoA21kUXO"

os.environ["CLIENT_ID"] = client_id
os.environ["CLIENT_SECRET"] = client_secret

## Using the SDK

Great now we're all set up to start using the SDK! Let's instantiate the Satellite Vu Client which will help us interact with our APIs!

In [67]:
import os
from satellitevu.client import Client

client_id = os.environ["CLIENT_ID"]
client_secret = os.environ["CLIENT_SECRET"]

client = Client(client_id=client_id, client_secret=client_secret)
client_id

'TFlrcIukpDcD2NqwykoL0hOAFzmtZWWC'

### Archive Searching

The API documentation for this can be found at https://api.qa.satellitevu.com/archive/v1/docs. For searching, all the parameters available in the API should work in the SDK. Although they may not be presented as defined args, you can still pass them in using **kwargs

In [68]:
from datetime import datetime, timedelta

# A search for the 100 most recent images in London over the past 2 months, sorted by most to least recent
search_params = {
    "bbox": [-1.065151, 51.163899, 0.457906, 51.802226], 
    "date_from": datetime.utcnow() - timedelta(days=60),
    "date_to": datetime.utcnow(),
    "limit" : 100,
    "sortby": [{"field": "datetime", "direction": "desc"}]
}

# Perform the search
london_search = client.archive_v1.search(**search_params)

# The search returns what the API does (i.e. a Response object from a web request) so we can check the status to see if it has been successful
if london_search.status != 200:
    raise Exception(f"Error: {london_search.status} - {london_search.text}")

search_results = london_search.json()

print (f"Total Results: {search_results['context']['matched']}") 
print (f"Returned results: {search_results['context']['returned']}")

stac_items = search_results["features"]
stac_items[0]

Total Results: 141
Returned results: 100


{'type': 'Feature',
 'geometry': {'coordinates': [[[-0.1819961150368847, 51.49543106733],
    [-0.2030703533053093, 51.49470818512379],
    [-0.2041498470340876, 51.47707392977891],
    [-0.1820247372977877, 51.47780314988645],
    [-0.1799446594695727, 51.48373425158267],
    [-0.17998843703230455, 51.489559043106354],
    [-0.1819961150368847, 51.49543106733]]],
  'type': 'Polygon'},
 'properties': {'created': '2022-10-07T08:39:49.387866+00:00',
  'updated': '2022-10-07T08:39:49.387866+00:00',
  'platform': 'tabi',
  'gsd': 3.5,
  'datetime': '2022-08-30T23:49:59+00:00',
  'proj:shape': [1280, 1024],
  'created_at': '2022-10-07T08:39:47Z',
  'eo:cloud_cover': 0,
  'proj:bbox': [694146.0, 5705451.0, 697730.0, 5709931.0],
  'proj:geometry': {'type': 'Polygon',
   'coordinates': [[[695607.25, 5708686.75],
     [694147.75, 5708550.25],
     [694147.75, 5706586.75],
     [695680.75, 5706726.75],
     [695799.75, 5707391.75],
     [695771.75, 5708039.25],
     [695607.25, 5708686.75]]]},
 

In [62]:
import json
import geopandas as gpd
import pandas as pd


for item in stac_items:
    coords = item["geometry"]["coordinates"]
    print (coords[0])
    break
# print(geomtries1)
# print(geomtries)

# stac_gdf = gpd.GeoDataFrame.from_features(search_results).set_index(pd.json_normalize(search_results["features"])["id"].values)
# stac_gdf.set_crs(epsg=32630, inplace=True)
# stac_gdf.to_crs(4326)
# stac_gdf.geometry.centroid.x


[[-0.1819961150368847, 51.49543106733], [-0.2030703533053093, 51.49470818512379], [-0.2041498470340876, 51.47707392977891], [-0.1820247372977877, 51.47780314988645], [-0.1799446594695727, 51.48373425158267], [-0.17998843703230455, 51.489559043106354], [-0.1819961150368847, 51.49543106733]]


In [ ]:
import folium

x1,y1,x2,y2 = stac_gdf['geometry'].total_bounds
print (x1,y1,x2,y2)
style = {'fillColor': '#228B22', 'color': '#228B22'}

map = folium.Map(location=[y1,x1], tiles='openstreetmap')

# map.fit_bounds([[y2, x1], [y1, x2]])
folium.GeoJson(stac_gdf["geometry"], style_function=lambda x:style).add_to(map)
map

### Orders and Downloads

The client can be used to programatically submit orders (of multiple items) and subsequently download them.

In [70]:
# Submitting an Order

stac_item_ids = [item["id"] for item in stac_items]
my_items = stac_item_ids[:5]
print (my_items)

my_order = client.orders_v1.submit(item_ids=my_items)

if my_order.status != 201:
    raise Exception(f"Error: {my_order.status} - {my_order.text}")

# Within this JSON you will receive an order ID. You will need this to download your imagery.
my_order_json = my_order.json()
my_order_id = my_order_json["id"]
print (f"Order ID: {my_order_id}")



['20220830T234959000_basic_0_TABI', '20220830T234958000_basic_0_TABI', '20220830T234957000_basic_0_TABI', '20220830T234956000_basic_0_TABI', '20220830T234955000_basic_0_TABI']
Order ID: 26d467e4-381a-4fba-b486-f1b277b4bc72


In [75]:
# Downloading an order

# Download the whole order - this will download all the imagery from that Order ID into a zip file at the location of your choice.
# The ZIP file will be name SatelliteVu_<order_id>.zip
# The path of this zip file is the return from the methof

downloaded_order = client.orders_v1.download_order(order_id=my_order_id, destdir="/Users/zhelinisivanesan/Downloads")
print (downloaded_order)


# Download an individal item in the order. The file will be named <item_id>.zip

download_item = client.orders_v1.download_item(order_id=my_order_id, item_id=my_items[0], destdir="/Users/zhelinisivanesan/Downloads")
print (download_item)

TypeError: argument should be integer or None, not 'bytes'